# Database API's voor relationele databanken

> Er bestaan verschillende methodes om vanuit een algemene programmeertaal (Java, C#, C++, Python, ...) 
> een relationele databank aan te spreken. 
> We bespreken elke methode aan de hand van Python als programmeertaal en 
> Micrsoft SQL Server als database, maar de principes zijn toepasbaar op 
> elke combinatie programmeertaal - databank.


## Voorbeeld relationele database: xtreme

![](./xtreme.png)

## Voorbeeldstatements

1. **Eenvoudige SELECT**  
   Toon productid, productname en voorraad (UnitsInStock) voor de 5 producten met de grootste vooraad. Sorteer aflopend op vooraad.
    ```sql
   select top 5 productid, productname, unitsinstock
   from product
   order by unitsinstock desc;
    ```
  
2. **Complexere SELECT**  
   Wat zijn de 10 best verkochte producten? Toon ook de omzet per product.
   
    ```sql
   select top 10 p.ProductID, p.ProductName, 
   sum(od.Quantity*od.UnitPrice) as sales
   from Product p join OrdersDetail od
   on p.ProductID=od.ProductID
   group by p.ProductID, p.ProductName
   order by sum(od.Quantity*od.UnitPrice) desc;
    ```

3. **UPDATE**  
   Verhoog de prijzen van alle producten van het producttype x met y % (x en y zijn parameters).

   ```sql
   update Product 
   set Price = Price * 1.10
   where ProductTypeID 
   in (select ProductTypeID 
   from Producttype where producttypename='Mountain')
   ``` 

4. **INSERT**  
   Voeg de niewe Courier "bpost" toe. 
    ```sql
   insert into Courier (CourierID, CourierName,Website)
   values (11,'bpost','www.bpost.be')
   ```

5. **DELETE**  
   Verwijder courier "bpost". 
   ```sql
   delete from Courier where CourierName = 'bpost'
    ```

## INHOUD
1. Plain SQL in Python code
2. SQL command in stored procedure
3. Language Integrated Query (LINQ)
4. Object Relational Mapping (ORM)


## 1. Plain SQL in Python code

- SQL is voor Python slechts een string
- Gebruik een database-specifieke Python-libary, bijv. pyodbc voor SQL Server of sqlite3 voor SQLite

### 1.1. Initiële setup

In [ ]:
!pip install pyodbc

In [ ]:
######################################
# Plain SQL: native PyODBC API  
######################################
print ('*** Native PyODBC API ***')

import pyodbc  

# connect to MS-SQL Server database using Windows authentication (Trusted_Connection=yes)
# connections strings are vendor specific!
conn = pyodbc.connect(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=NB22-DMTYCL3;DATABASE=xtreme;Trusted_Connection=yes;')
cursor = conn.cursor()




### 1.2. Statement 1

```sql
   select top 5 productid, productname, unitsinstock
   from product
   order by unitsinstock desc;
```

In [ ]:
stmt = "select top 5 productid, productname, unitsinstock from product order by unitsinstock desc;"
cursor.execute(stmt)
rows = cursor.fetchall()
print(f'Rows is of data type {type(rows)}')
print()

print(f'{"ID":<5} {"Name":<30} {"Units in stock":>7}')
print('-'*51)
for row in rows:
    print(f'{row[0]:<5} {row[1]:<30} {round(row[2]):>7}')



### 1.3. Statement 2

```sql
select top 10 p.ProductID, p.ProductName, 
sum(od.Quantity*od.UnitPrice) as sales
from Product p join OrdersDetail od
on p.ProductID=od.ProductID
group by p.ProductID, p.ProductName
order by sum(od.Quantity*od.UnitPrice) desc;
```

In [ ]:
stmt =  """
        select top 10 p.ProductID, p.ProductName, 
        sum(od.Quantity*od.UnitPrice) as sales
        from Product p join OrdersDetail od
        on p.ProductID=od.ProductID
        group by p.ProductID, p.ProductName
        order by sum(od.Quantity*od.UnitPrice) desc;
        """

cursor.execute(stmt)
rows = cursor.fetchall()

print(f'{"ID":<10} {"Name":<30} {"Sales":>7}')
print('-'*51)
for row in rows:
    print(f'{row[0]:<10} {row[1]:<30} {round(row[2],2):7.2f}' )

### 1.4. Statement 3

```sql
update Product 
set Price = Price * 1.10
where ProductTypeID 
in (select ProductTypeID 
from Producttype where producttypename='Mountain')
```

In [ ]:
# Statement 3

stmt = """
    update Product 
    set Price = Price * (1 + ?/100.0)
    where ProductTypeID 
    in (select ProductTypeID 
    from Producttype where producttypename=?);
    """
cursor.execute(stmt, (10, 'Mountain') )

In [ ]:
# Check results
stmt = "select productid, productname,price from product \
        where ProductTypeID in \
        (select ProductTypeID from Producttype where producttypename='Mountain');"
cursor.execute(stmt)
rows = cursor.fetchall()

print(f'{"ID":<6} {"Name":<15} {"Price":<7}')
print('-'*30)
for row in rows:
    print(f'{row[0]:<6} {row[1]:<15} {round(row[2],2):7.2f}' )


Ofwel, updates terugdraaien...

In [11]:
conn.rollback() # by default autocommit is off

...ofwel, updates bevestigen. 

In [8]:
conn.commit()

### 1.5 Statement 4
```sql
insert into Courier (CourierID, CourierName,Website)
values (11,'bpost','www.bpost.be')
```

In [ ]:
stmt = "insert into Courier (CourierID, CourierName,Website) values (11,'bpost','www.bpost.be');"
cursor.execute(stmt )

In [ ]:
# Check results
stmt = "SELECT * FROM courier;"
cursor.execute(stmt)
rows = cursor.fetchall()

print(f'{"CourierID":<10} {"Name":<30}')
print('-'*26)
for row in rows:
    print(f'{row[0]:<10} {row[1]:<30}' )

Voer `rollback()` uit om de wijzigingen terug te draaien, of `commit()` om ze definitief te maken. 

In [15]:
conn.rollback()

In [12]:
conn.commit()

### 1.6 Statement 5
```sql
delete from Courier where CourierName = 'bpost'
```

In [ ]:
stmt = "delete from Courier where CourierName = 'bpost';"
cursor.execute(stmt )

In [ ]:
# Check results
stmt = "SELECT * FROM courier ORDER BY courierid;"
cursor.execute(stmt)
rows = cursor.fetchall()

print(f'{"CourierID":<10} {"Name":<15}')
print('-'*26)
for row in rows:
    print(f'{row[0]:<10} {row[1]:<15}')

In [15]:
conn.rollback()

In [21]:
conn.commit()

### 1.7. Afsluiten

In [22]:
conn.close()

### 1.8. Voor- en nadelen van "Plain SQL"

- `+` Eenvoudig in gebruik: test je SQL string m.b.v. een SQL tool en copy-paste naar Python
- `-` Geen SQL syntax check bij de ontwikkeling --> errors duiken op at runtime
- `-` Geen integratie met Python objecten.
- `-` Code is niet porteerbaar naar andere databanken: 
  - `-` Je gebruikt software-bibliotheken die specifiek zijn voor een bepaald database-systeem (= native API)
  - `-` SQL-code volgt dialect van een bepaalde database en is dus niet noodzakelijk porteerbaar naar een andere database (vb. `SELECT TOP ...`). 

## 2. SQL command in stored procedure

- Syntactisch identiek aan "Plain SQL".
- SQL command wordt ingebed in stored procedure ("embedded SQL") en buiten Python getest.
- In Python enkel nog "exec <stored proc>". 

Omdat de werkwijze telkens dezelfde is, illustreren we dit enkel aan de hand van Statement 1. 

```sql
select top 5 productid, productname, unitsinstock
from product
order by unitsinstock desc;
```

In [23]:
# connect to MS-SQL Server database using Windows authentication (Trusted_Connection=yes)
# connections strings are vendor specific!
conn = pyodbc.connect(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=NB22-DMTYCL3;DATABASE=xtreme;Trusted_Connection=yes;')
cursor = conn.cursor()

In [ ]:
'''
In database xtreme: 
    create or alter procedure ProductsInStock
    as
        select top 5 productid, productname, unitsinstock from product order by unitsinstock desc;
'''

stmt = "exec ProductsInStock;"
cursor.execute(stmt)
rows = cursor.fetchall()
print(f'Rows is of data type {type(rows)}')
print()

print(f'{"ID":<5} {"Name":<30} {"Units in stock":>7}')
print('-'*51)
for row in rows:
    print(f'{row[0]:<5} {row[1]:<30} {round(row[2]):>7}')

### SQL command in stored procedure
- `+` Eenvoudig in gebruik: test je SQL string m.b.v. een SQL tool, steek die in een procedure en voor enkel 'exec <proc>' uit in Python. 
- `+` SQL syntax check bij de ontwikkeling bij CREATE/ALTER STORED PROCEDURE
- `-` Geen integratie met Python objecten.
- `-` Code is niet porteerbaar naar andere databanken: 
- `-` Je gebruikt software-bibliotheken die specifiek zijn voor een bepaald database-systeem (= native API).

## 3. Language Integrated Query (LINQ)

- Bouw het SQL-commando op aan de hand van Python-functies. 
- Gebruik een database-onafhankelijke Python-library: SQL Alchemy Core. 
- Tabellen als geheel zijn objecten. 

In [ ]:
!pip install sqlalchemy

### 3.1. Initiële setup

In [ ]:
###################################################################################
#  LINQ (Language Integrated Query) met SQL Alchemy Core --> 
###################################################################################

import sqlalchemy
sqlalchemy.__version__

In [ ]:
# Establishing a connection
print ('*** SQL Alchemy Core ***')

from sqlalchemy import create_engine  
engine = create_engine('mssql+pyodbc://NB22-DMTYCL3/xtreme?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

conn = engine.connect() 

In [28]:
# Setting up MetaData with Table Objects
from sqlalchemy import MetaData, Table, Column, Integer, String, Float, Numeric
metadata = MetaData()   # This object is essentially a facade around a Python dictionary 
                        # that stores a series 
                        # of Table objects keyed to their string name. 

# Option 1: explicit Table objects
Product = Table('Product', metadata, 
               Column('ProductID', Integer, primary_key=True),
               Column('ProductName', String),
               Column('Color', String),
               Column('Sizes', String),
               Column('M_F', String),
               Column('Price', Numeric(8,2)),
               Column('ProductTypeID', Integer),
               Column('ProductClassID', Integer),
               Column('SupplierID', Integer),
               Column('ReorderLevel', Integer),
               Column('UnitsInStock', Integer))          

# Option 2: reflecting tables: generate Table objects automatically from database     
OrdersDetail = Table('OrdersDetail', metadata, autoload_with=engine)
ProductType = Table('ProductType', metadata, autoload_with=engine)
Courier = Table('Courier', metadata, autoload_with=engine)  

### 3.2. Statement 1

```sql
select top 5 productid, productname, unitsinstock
from product
order by unitsinstock desc;
```

In [ ]:
from sqlalchemy import select
stmt = select(Product.c.ProductID,Product.c.ProductName, Product.c.UnitsInStock) \
       .order_by(Product.c.UnitsInStock.desc()) \
       .limit(5)

rows = conn.execute(stmt)

print(f'{"ID":<5} {"Name":<30} {"Units in stock":>7}')
print('-'*51)
for row in rows:
    print(f'{row[0]:<5} {row[1]:<30} {round(row[2]):>7}')


### 3.3. Statement 2

```sql
select top 10 p.ProductID, p.ProductName, 
sum(od.Quantity*od.UnitPrice) as sales
from Product p join OrdersDetail od
on p.ProductID=od.ProductID
group by p.ProductID, p.ProductName
order by sum(od.Quantity*od.UnitPrice) desc;
```

In [ ]:
from sqlalchemy import select, func

stmt = select(Product.c.ProductID,Product.c.ProductName,func.sum(OrdersDetail.c.Quantity * OrdersDetail.c.UnitPrice)) \
       .select_from(Product) \
       .join(OrdersDetail, Product.c.ProductID == OrdersDetail.c.ProductID) \
       .group_by(Product.c.ProductID, Product.c.ProductName) \
       .order_by(func.sum(OrdersDetail.c.Quantity * OrdersDetail.c.UnitPrice).desc()) \
       .limit(10)

print(stmt)

rows = conn.execute(stmt)

print(f'{"ID":<10} {"Name":<30} {"Sales":>7}')
print('-'*51)
for row in rows:
    print(f'{row[0]:<10} {row[1]:<30} {round(row[2],2):7.2f}' )

### 3.4. Statement 3

```sql
update Product 
set Price = Price * 1.10
where ProductTypeID 
in (select ProductTypeID from Producttype where producttypename='Mountain')
```

In [ ]:
from sqlalchemy import update,bindparam

subq = select(ProductType.c.ProductTypeID).where(ProductType.c.ProductTypeName == bindparam("type"))

stmt = (update(Product).values(Price=Product.c.Price * (1 + bindparam("pct"))).where(Product.c.ProductTypeID.in_(subq)))
print(stmt) # string representation of the statement
conn.execute(stmt,{"pct":0.1, "type":"Mountain"}) 


In [ ]:
# Check results
from sqlalchemy import select

subq = select(ProductType.c.ProductTypeID).where(ProductType.c.ProductTypeName == "Mountain")

stmt = select(Product.c.ProductID,Product.c.ProductName,Product.c.Price) \
        .where(Product.c.ProductTypeID.in_(subq))
print(stmt)
rows = conn.execute(stmt)

print(f'{"ID":<5} {"Name":<40} {"Unitprice":>7}')
print('-'*54)
for row in rows:
    print(f'{row[0]:<5} {row[1]:<40} {row[2]:7.2f}')
    

In [33]:
conn.rollback()

In [29]:
conn.commit()

### 3.5 Statement 4
```sql
insert into Courier (CourierID, CourierName,Website)
values (11,'bpost','www.bpost.be')
```

In [ ]:
from sqlalchemy import insert
stmt = insert(Courier).values(CourierID=11,CourierName='bpost', Website='www.bpost.be')
conn.execute(stmt)

In [ ]:
# Check results
from sqlalchemy import select

stmt = select(Courier)
rows = conn.execute(stmt)

print(f'{"CourierID":<10} {"CourierName":<30} {"Website":<30}')
print('-'*72)
for row in rows:
    print(f'{row[0]:<10} {row[1]:<30} { row[2] if row[2] else ""}' )

In [40]:
conn.rollback()

In [56]:
conn.commit()

### 3.6 Statement 5
```sql
delete from Courier where CourierName = 'bpost''
```

In [ ]:
from sqlalchemy import delete
stmt = delete(Courier).where(Courier.c.CourierName == 'bpost')
conn.execute(stmt)

In [ ]:
# Check results
from sqlalchemy import select

stmt = select(Courier)
rows = conn.execute(stmt)

print(f'{"CourierID":<10} {"CourierName":<30} {"Website":<30}')
print('-'*72)
for row in rows:
    print(f'{row[0]:<10} {row[1]:<30} { row[2] if row[2] else ""}' )

In [60]:
conn.rollback()

In [37]:
conn.commit()

### 3.7. Afsluiten

In [62]:
conn.close()

### 3.8. Voor- en nadelen van LINQ
- `+` SQL syntax check door Python --> fouten ontdekt bij het ontwikkelen
- `+` Porteerbaar tussen databanksystemen
- `-` Geen integratie met Python-objecten.
- `-` Extra syntax moet aangeleerd worden. 

## 4. Object Relational Mapping (ORM)

- Werk volledig op basis van Python-objecten. 
- Gebruik een database-onafhankelijke Python-library: SQL Alchemy ORM

### 4.1. Initiële setup

In [ ]:
###################################################################################
# Database independant + application objects --> SQL Alchemy ORM 
###################################################################################

# Establishing a connection
print ('*** SQL Alchemy ORM ***')

from sqlalchemy import create_engine  
from sqlalchemy.orm import Session

engine = create_engine('mssql+pyodbc://NB22-DMTYCL3/xtreme?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

conn = engine.connect() 

session = Session(engine)


Hier definiëren we constructies op moduleniveau die de structuren zullen vormen die we zullen bevragen vanuit de database.   
Deze structuur, bekend als een Declarative Mapping, definieert zowel een Python objectmodel,   
als database metadata die echte SQL tabellen beschrijft die bestaan, of zullen bestaan, in een bepaalde database. 

In [77]:
from sqlalchemy import String, select
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column

from decimal import Decimal
from typing_extensions import Annotated
from sqlalchemy import Numeric
from sqlalchemy.orm import registry
num_8_2 = Annotated[Decimal, 8]

class Base(DeclarativeBase):
    registry = registry(
        type_annotation_map={
            num_8_2: Numeric(6, 2)
        }
    )

# Option 1: explicit Table objects
       
class Product(Base):
    __tablename__ = "Product"
    ProductID: Mapped[int] = mapped_column(primary_key=True)
    ProductName: Mapped[str]
    Color: Mapped[str]
    Sizes: Mapped[str]
    M_F: Mapped[str]
    Price: Mapped[float]
    ProductTypeID: Mapped[int]
    ProductClassID: Mapped[int]
    SupplierID: Mapped[int]
    ReorderLevel: Mapped[int]
    UnitsInStock: Mapped[int]

# Option 2: reflecting tables: generate Table objects automatically from database   

Base.metadata.reflect(engine)   # get metadata from database

class OrdersDetail(Base):  # each table is a subclass from the Base table
    __table__ = Base.metadata.tables['OrdersDetail']

class ProductType(Base):  
    __table__ = Base.metadata.tables['ProductType']
  
class Courier(Base):
    __table__ = Base.metadata.tables['Courier']




### 4.2. Statement 1

```sql
select top 5 productid, productname, unitsinstock
from product
order by unitsinstock desc;
```

In [ ]:
stmt = select(Product.ProductID,Product.ProductName, (Product.UnitsInStock).label('Stock')) \
        .order_by(Product.UnitsInStock.desc()) \
        .limit(5)

products = session.execute(stmt)

print(f'{"ID":<5} {"Name":<30} {"Stock":>7}')
print('-'*44)

for product in products:
    print(f'{product.ProductID:<5} {product.ProductName:<30} {product.Stock:>7}')

### 4.3. Statement 2

```sql
select top 10 p.ProductID, p.ProductName, 
sum(od.Quantity*od.UnitPrice) as sales
from Product p join OrdersDetail od
on p.ProductID=od.ProductID
group by p.ProductID, p.ProductName
order by sum(od.Quantity*od.UnitPrice) desc;
```

In [ ]:
from sqlalchemy import func

stmt = select(Product.ProductID,Product.ProductName,func.sum(OrdersDetail.Quantity * OrdersDetail.UnitPrice).label('Sales')) \
        .select_from(Product) \
        .join(OrdersDetail, Product.ProductID == OrdersDetail.ProductID) \
        .group_by(Product.ProductID, Product.ProductName) \
        .order_by(func.sum(OrdersDetail.Quantity * OrdersDetail.UnitPrice).desc()) \
        .limit(10)

rows = session.execute(stmt)

print(f'{"ID":<10} {"Name":<30} {"Sales":>7}')
print('-'*49)
for row in rows:
    print(f'{row.ProductID:<10} {row.ProductName:<30} {round(row.Sales,2):7.2f}' )



### 4.4. Statement 3

```sql
update Product 
set Price = Price * 1.10
where ProductTypeID 
in (select ProductTypeID 
from Producttype where producttypename='Mountain')
```

In [ ]:
# Option 1: with update statement
from sqlalchemy import update, bindparam
stmt = update(Product).values(Price=Product.Price * (1 + bindparam("pct"))).where(Product.ProductTypeID.in_(select(ProductType.ProductTypeID).where(ProductType.ProductTypeName == bindparam("type"))))
print(stmt) # stringify the statement
session.execute(stmt,{"pct":0.1, "type":"Mountain"}) 


In [ ]:
# Option 2: with ORM objects
pct = 0.1
type = 'Mountain'

stmt = select(Product).where(Product.ProductTypeID.in_(select(ProductType.ProductTypeID).where(ProductType.ProductTypeName == type))) 
print(stmt)  # stringify the statement
products = session.scalars(stmt)  # with scalars() we receive ORM entities directly

for product in products:
    product.Price = product.Price * (1 + pct)


In [ ]:
# Check results
stmt = select(Product).where(Product.ProductTypeID.in_(select(ProductType.ProductTypeID).where(ProductType.ProductTypeName == 'Mountain')))
print(stmt)  # stringify the statement
products = session.scalars(stmt)  # with scalars() we receive ORM entities directly

print(f'{"ID":<5} {"Name":<40} {"price":>7}')
print('-'*55)
for product in products:
    print(f'{product.ProductID:<5} {product.ProductName:<40} {product.Price:>7.2f}')

In [72]:
session.rollback()

In [47]:
session.commit()

### 4.5 Statement 4
```sql
insert into Courier (CourierID, CourierName,Website)
values (11,'bpost','www.bpost.be')
```

- Bij gebruik van het ORM is het Session-object verantwoordelijk voor het construeren van Insert-constructies en het uitzenden ervan in een transactie. 
- De manier waarop we de Session instrueren dit te doen is door object entries toe te voegen.
- De Session zorgt er dan voor dat deze nieuwe entries naar de database worden verzonden wanneer ze nodig zijn, met behulp van een proces dat bekend staat als een flush.



In [78]:
bpost = Courier(CourierID=11, CourierName='bpost', Website='www.bpost.be')
session.add(bpost)

In [ ]:
# Check results
stmt = select(Courier)
print(stmt)  # stringify the statement
couriers = session.scalars(stmt)

print(f'{"CourierID":<10} {"CourierName":<30} {"Website":<30}')
print('-'*72) 
for courier in couriers:
    print(f'{courier.CourierID:<10} {courier.CourierName:<30} {courier.Website if courier.Website else ""}' )

In [80]:
session.rollback()

In [51]:
session.commit()

### 4.6 Statement 5
```sql
delete from Courier where CourierName = 'bpost'
```

In [ ]:
stmt = select(Courier).where(Courier.CourierName == 'bpost')
result = session.scalars(stmt).all()  # with scalars() we receive ORM entities directly,  with all() we get all results as a list
if len(result) > 0:  
    print('Courier bpost exists')
    bpost = result[0]   # get first courier with name bpost
    session.delete(bpost)
    print('Courier bpost deleted')
else:
    print('Courier bpost does not exist')


In [ ]:
# Check results
stmt = select(Courier)
print(stmt)  # stringify the statement
couriers = session.scalars(stmt)

print(f'{"CourierID":<10} {"CourierName":<30} {"Website":<30}')
print('-'*72) 
for courier in couriers:
    print(f'{courier.CourierID:<10} {courier.CourierName:<30} {courier.Website if courier.Website else ""}' )

In [54]:
session.rollback()

In [55]:
session.commit()

### 4.7. Afsluiten

In [75]:
conn.close()

### 4.8. Voor- en nadelen van ORM
- `+` SQL syntax check door Python --> fouten ontdekt bij het ontwikkelen
- `+` Porteerbaar tussen databanksystemen
- `+` Integratie met Python-objecten
- `-` Extra syntax moet aangeleerd worden
- `-` Risico op trage code (vooral bij Optie 2: rechtstreeks gebruik van objecten) omdat men niet meer stilstaat bij gegenereerde SQL-commando's: 
  - SELECT *: onnodig veel kolommen ophalen
  - teveel "round-trips" naar de database, door bijv. in een lus updates uit te voeren i.p.v. in één update-statement

## 4. Conclusies

- Gebruik "Plain SQL" 
  - om snel resultaat te hebben
  - als je vertrouwd bent met de SQL-taal
  - voor erg complexe query's die moeilijk om te zetten zijn naar SQLAlchemy
  - maar geef er wel de voorkeur aan om de query via een stored procedure uit te voeren
- Gebruik SQL Alchemy (ORM of Core)
  - voor productiewaardige software
- Gebruik SQL Alchemy ORM
  - als object-oriëntatie een must is 
  - als onderdeel van een grotere applicatie
  - als "state" of geheugen tussen calls belangrijk is
  - maar trap niet in de "performantie-val": zorg ervoor dat je goed weet wat er gebeurt op de database